Sequential Tool Calls

The LLM decides when to call another tool and which tool to call next in a seq task instead of hardcoding the sequence

LLM => Tool Call => State Update => LLM

In [ ]:
# Tools
def multiply(a: int, b: int) -> int:
    """Multiply two integers and return the product."""
    return a * b

def translate_to_french(text: str) -> str:
    """Return a mock French translation of the input text."""
    return f"French translation of '{text}'"

def get_weather(city: str) -> str:
    """Return a mock weather string for the given city."""
    return f"The weather in {city} is sunny."

Each tool needs a short docstring so LangGraph can wrap it correctly as a StructuredTool

LangGraph WorkFlow

In [ ]:
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_openai import ChatOpenAI

In [ ]:
# State that holds conversation history
class MyMessagesState(MessagesState):
    pass

In [ ]:
# LLM with all tools bound
llm = ChatOpenAI(model="gpt-4o")
llm_with_tools = llm.bind_tools([multiply, translate_to_french, get_weather])

LLM node: Listens to the conversation and may produce a tool call.

In [ ]:
# Node: AI generates a response and may request a tool
def tool_calling_llm(state: MyMessagesState):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

In [ ]:
# Build the graph
builder = StateGraph(MyMessagesState)

In [ ]:
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode([multiply, translate_to_french, get_weather]))

In [ ]:
# Flow: START -> LLM
builder.add_edge(START, "tool_calling_llm")

In [ ]:
# LLM -> tools OR END
builder.add_conditional_edges("tool_calling_llm", tools_condition, {"tools": "tools", "default": END})

In [ ]:
# After tool execution, return to LLM for possible next step
builder.add_edge("tools", "tool_calling_llm")

In [ ]:
graph = builder.compile()

Inference

In [ ]:
from langchain_core.messages import HumanMessage

# Multi-step example: requires get_weather THEN translate_to_french
messages = graph.invoke({"messages": [HumanMessage(content="Translate the weather in Paris into French")]})
for m in messages["messages"]:
    m.pretty_print()

In [ ]:
from langchain_core.messages import HumanMessage

# Single-step example
messages = graph.invoke({"messages": [HumanMessage(content="Multiply 2 and 3")]})
for m in messages["messages"]:
    m.pretty_print()